# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Batch Processing** </center>
---

**Date**: October, 2025

**Student Name**: Vicente Sebastian Serrano Cabrera

**Professor**: Pablo Camarillo Ramirez

# Introduction

## Introducción

En juegos competitivos como *Valorant*, se generan múltiples eventos en tiempo real durante cada partida: movimientos, disparos, habilidades, objetivos, etc. Manejar esta gran cantidad de datos simultáneos es un reto, ya que si no se procesan y almacenan correctamente, se pierde información valiosa para el análisis del rendimiento, mejoras del matchmaking y detección de trampas.

El objetivo de este proyecto es implementar una parte del pipeline propuesto previamente: limpiar y transformar datos de telemetría con Apache Spark, y posteriormente persistir la información procesada en una base de datos PostgreSQL. Con esto se busca asegurar que los datos estén organizados, completos y disponibles para futuros análisis y visualizaciones.


In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Final Project: Batch Processing") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", "/opt/spark/work-dir/jars/postgresql-42.7.8.jar") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
spark.conf.set("spark.sql.shuffle.partitions", "5")

25/10/22 00:57:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Dataset

In [2]:
valorant_df = spark.read \
                .option("header", "true") \
                .csv("/opt/spark/work-dir/data/valorant_events.csv", header=True, inferSchema=True)

valorant_df.show(5)

valorant_df.printSchema()

+--------------------+--------+---------+----------------+-------+--------+---------+----------+-----------------+--------------------+
|            event_id|match_id|player_id|     player_name|   rank|map_name|game_mode|event_type|weapon_or_ability|           timestamp|
+--------------------+--------+---------+----------------+-------+--------+---------+----------+-----------------+--------------------+
|94a364f7-9e19-4a1...|   M8401|    P1561|     grantmelton|   Iron|   Haven|SpikeRush|     Death|         Operator|2025-09-19 07:25:...|
|16a57989-681f-4e9...|   M7987|    P1118|       noahbowen|   Iron|  Breeze|  Unrated|SpikePlant|       HealingOrb|2025-09-29 01:03:...|
|a38f1989-bb71-4ab...|   M8183|    P1852| darrellmarshall|   Iron|    Bind|SpikeRush|SpikePlant|          Classic|2025-09-27 11:18:...|
|e2f3a46a-91a8-4ad...|   M7639|    P1328|         vjordan|Diamond|   Split|SpikeRush|SpikePlant|          Grenade|2025-09-21 06:04:...|
|53af9784-9634-42b...|   M7246|    P1443|carpent

# Transformations and Actions

In [8]:
from pyspark.sql.functions import col, to_date, hour, when, count, avg

valorant_df = valorant_df.withColumn("event_date", to_date(col("timestamp"))) \
                         .withColumn("event_hour", hour(col("timestamp"))) \
                         .withColumn("critical_event", 
                                     when(col("event_type").isin(["Kill", "SpikePlant", "SpikeDefuse"]), 1)
                                     .otherwise(0))

valorant_df_clean = valorant_df.dropna(subset=["player_id", "match_id", "event_type", "timestamp"]) \
                               .dropDuplicates()

df_player_stats = valorant_df_clean.groupBy("player_id", "player_name", "rank") \
                                   .agg(count("critical_event").alias("total_critical_events"))

df_map_stats = valorant_df_clean.groupBy("map_name") \
                                .agg(avg("critical_event").alias("avg_critical_events"))




# Persistence Data

### Justificación de la selección de la base de datos

Para la persistencia de los datos transformados se seleccionó PostgreSQL debido a 2 motivos, el primero es que fue el usado duarente las clases y el conocido para implementar, mientras que el segundo motivo es que el modelo de información del proyecto está altamente estructurado y presenta relaciones claras entre entidades como jugadores, partidas, mapas y eventos dentro del juego Valorant. Este tipo de datos se adapta mejor a un esquema relacional, donde la consistencia, normalización y capacidad para realizar consultas analíticas son esenciales.

PostgreSQL permite:
- Mantener integridad referencial entre los datos de telemetría procesados.
- Ejecutar consultas SQL complejas para obtener métricas del desempeño de jugadores o balance de mapas.
- Escalar el almacenamiento sin perder consistencia y fiabilidad en la información.
- Integrarse fácilmente con Spark mediante conexión JDBC para cargas y actualizaciones periódicas.




In [9]:
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"
db_properties = {
    "user": "postgres",
    "password": "Admin@1234",
    "driver": "org.postgresql.Driver"
}

valorant_df_clean.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "valorant_clean") \
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .mode("overwrite") \
    .save()

print("valorant_clean guardado")

df_player_stats.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "valorant_player_stats") \
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .mode("overwrite") \
    .save()

print("valorant_player_stats guardado")

df_map_stats.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "valorant_map_stats") \
    .option("user", db_properties["user"]) \
    .option("password", db_properties["password"]) \
    .option("driver", db_properties["driver"]) \
    .mode("overwrite") \
    .save()

print("valorant_map_stats guardado")


valorant_clean guardado correctamente.
valorant_player_stats guardado correctamente.
valorant_map_stats guardado correctamente.


In [10]:
for t in ["valorant_clean", "valorant_player_stats", "valorant_map_stats"]:
    print(f"\nTabla: {t}")
    df_check = spark.read.jdbc(url=jdbc_url, table=t, properties=db_properties)
    df_check.show(5, truncate=False)


Tabla: valorant_clean
+------------------------------------+--------+---------+----------------+--------+--------+---------+-----------+-----------------+--------------------------+----------+----------+--------------+
|event_id                            |match_id|player_id|player_name     |rank    |map_name|game_mode|event_type |weapon_or_ability|timestamp                 |event_date|event_hour|critical_event|
+------------------------------------+--------+---------+----------------+--------+--------+---------+-----------+-----------------+--------------------------+----------+----------+--------------+
|a38f1989-bb71-4ab8-8c83-3563ce6dd900|M8183   |P1852    |darrellmarshall |Iron    |Bind    |SpikeRush|SpikePlant |Classic          |2025-09-27 11:18:18.772287|2025-09-27|11        |1             |
|53af9784-9634-42bd-b9ef-e2309642cb70|M7246   |P1443    |carpenterjeffrey|Bronze  |Bind    |SpikeRush|SpikePlant |Grenade          |2025-09-21 21:52:28.278424|2025-09-21|21        |1       

# DAG

In [ ]:
sc.stop()